In [5]:
%%html

<style>
th {
    text-align: left;
}
</style>

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
from depsurf import LinuxImage

img1 = LinuxImage.from_str("5.4.0-26-generic-amd64")
img2 = LinuxImage.from_str("5.8.0-25-generic-amd64")

In [8]:
import pandas as pd

from depsurf import FileLogger, check_diff, OUTPUT_PATH, Kind, LinuxImage


def diff_btf(img1: LinuxImage, img2: LinuxImage, group, name):
    path = OUTPUT_PATH / group / name
    print(f"Results saving to {path}")

    results = {}
    for key, kind in [
        ("func", Kind.FUNC),
        ("struct", Kind.STRUCT),
        # ("union", Kind.UNION),
        # ("enum", Kind.ENUM),
        ("tp_event", Kind.STRUCT),
        ("tp_func", Kind.FUNC),
        ("lsm", Kind.FUNC),
    ]:
        if key == "lsm":
            dict1 = img1.lsm_hooks
            dict2 = img2.lsm_hooks
        elif key == "tp_func":
            dict1 = img1.tracepoints.funcs
            dict2 = img2.tracepoints.funcs
        elif key == "tp_event":
            dict1 = img1.tracepoints.events
            dict2 = img2.tracepoints.events
        else:
            dict1 = img1.btf.get_all_kind(kind)
            dict2 = img2.btf.get_all_kind(kind)

        output_path = path / f"{key}.log"
        with FileLogger(output_path):
            diff_result = check_diff(dict1, dict2, kind)
        results[(key, "Old")] = len(dict1)
        results[(key, "New")] = len(dict2)
        results[(key, "Common")] = len(diff_result.common)
        results[(key, "Added")] = len(diff_result.added)
        results[(key, "Removed")] = len(diff_result.removed)
        results[(key, "Changed")] = len(diff_result.changed)
        for k, v in diff_result.reasons.items():
            results[(key, f"- {k}")] = v

    df = pd.DataFrame(
        results.values(),
        index=pd.MultiIndex.from_tuples(results.keys()),
        columns=[name],
    )
    df.to_string(path / "summary.txt")
    df.to_pickle(path / "summary.pkl")
    return df


# diff_btf(img1, img2, "dist", "test")

In [9]:
from depsurf import BuildVersion

lts_imgs = [bv.img for bv in BuildVersion.iter(flavor="generic", arch="amd64", is_lts=True)]
lts_imgs

[LinuxImage(4.4.0-21-generic-amd64),
 LinuxImage(4.15.0-20-generic-amd64),
 LinuxImage(5.4.0-26-generic-amd64),
 LinuxImage(5.15.0-25-generic-amd64),
 LinuxImage(6.8.0-22-generic-amd64)]

In [10]:
all_imgs = [bv.img for bv in BuildVersion.iter(flavor="generic", arch="amd64")]
all_imgs

[LinuxImage(4.4.0-21-generic-amd64),
 LinuxImage(4.8.0-22-generic-amd64),
 LinuxImage(4.10.0-19-generic-amd64),
 LinuxImage(4.13.0-16-generic-amd64),
 LinuxImage(4.15.0-20-generic-amd64),
 LinuxImage(4.18.0-10-generic-amd64),
 LinuxImage(5.0.0-13-generic-amd64),
 LinuxImage(5.3.0-18-generic-amd64),
 LinuxImage(5.4.0-26-generic-amd64),
 LinuxImage(5.8.0-25-generic-amd64),
 LinuxImage(5.11.0-16-generic-amd64),
 LinuxImage(5.13.0-19-generic-amd64),
 LinuxImage(5.15.0-25-generic-amd64),
 LinuxImage(5.19.0-21-generic-amd64),
 LinuxImage(6.2.0-20-generic-amd64),
 LinuxImage(6.5.0-9-generic-amd64),
 LinuxImage(6.8.0-22-generic-amd64)]

In [11]:
revision_imgs = [bv.img for bv in BuildVersion.iter(flavor="generic", arch="amd64", revision="all", version="5.4.0")]

revision_imgs

[LinuxImage(5.4.0-26-generic-amd64),
 LinuxImage(5.4.0-51-generic-amd64),
 LinuxImage(5.4.0-75-generic-amd64),
 LinuxImage(5.4.0-100-generic-amd64),
 LinuxImage(5.4.0-125-generic-amd64),
 LinuxImage(5.4.0-175-generic-amd64)]

In [12]:
from itertools import pairwise

results = {}

def diff_images(group, imgs):
    for img1, img2 in pairwise(imgs):
        print(f"Comparing {img1} and {img2}")
        name = f"{img1.version}|{img2.version}"
        df = diff_btf(img1, img2, "dist", name)
        results[(group, name)] = df


diff_images("lts", lts_imgs)
diff_images("all", all_imgs)
diff_images("revision", revision_imgs)


df = pd.concat(results, axis=1)
df.columns = df.columns.droplevel(1)
df.to_pickle("output/diff.pkl")

df

[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/4.4.0-21-generic-amd64.pkl


Comparing LinuxImage(4.4.0-21-generic-amd64) and LinuxImage(4.15.0-20-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.4.0-21-generic-amd64|4.15.0-20-generic-amd64


[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/4.15.0-20-generic-amd64.pkl
[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.4.0-21-generic-amd64.pkl
[tracepoint.py:66 ] WARNING: Could not find function for xen_mmu_flush_tlb_all
[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.15.0-20-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-26-generic-amd64.pkl


Comparing LinuxImage(4.15.0-20-generic-amd64) and LinuxImage(5.4.0-26-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/4.15.0-20-generic-amd64|5.4.0-26-generic-amd64


[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.15.0-25-generic-amd64.pkl


...

Comparing LinuxImage(5.4.0-100-generic-amd64) and LinuxImage(5.4.0-125-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/5.4.0-100-generic-amd64|5.4.0-125-generic-amd64


[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-125-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-175-generic-amd64.pkl


Comparing LinuxImage(5.4.0-125-generic-amd64) and LinuxImage(5.4.0-175-generic-amd64)
Results saving to /Users/szhong/Downloads/bpf-study/btf/output/dist/5.4.0-125-generic-amd64|5.4.0-175-generic-amd64


[  objfile.py:34 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-175-generic-amd64.pkl


lts  \
                               4.4.0-21-generic-amd64|4.15.0-20-generic-amd64   
func     Old                                                            36309   
         New                                                            42299   
         Common                                                         33699   
         Added                                                           8600   
         Removed                                                         2610   
         Changed                                                         1956   
         - Param added                                                   1115   
         - Param removed                                                  800   
         - Param type changed                                             508   
         - Param reordered                                                375   
         - Return type changed                                            292   
struct   Old                                                             6160   
         New                                                             7342   
         Common                                                          5893   
         Added                                                           1449   
         Removed                                                          267   
         Changed                                                         1364   
         - Field added                                                    798   
         - Field removed                                                  454   
         - Field type changed                                             416   
         - Layout changed                                                   0   
tp_event Old                                                              502   
         New                                                              675   
         Common                                                           478   
         Added                                                            197   
         Removed                                                           24   
         Changed                                                           40   
         - Field added                                                     28   
         - Field removed                                                   26   
         - Field type changed                                               3   
         - Layout changed                                                   0   
tp_func  Old                                                              502   
         New                                                              675   
         Common                                                           478   
         Added                                                            197   
         Removed                                                           24   
         Changed                                                           22   
         - Param added                                                     13   
         - Param removed                                                    8   
         - Param type changed                                               3   
         - Param reordered                                                  4   
         - Return type changed                                              0   
lsm      Old                                                              168   
         New                                                              177   
         Common                                                           162   
         Added                                                             15   
         Removed                                                            6   
         Changed                                                           19   
         - Param added  